In [2]:
import json
import numpy as np

# Paso 1: Crear un diccionario para mapear cada campeón con sus estadísticas

# Cargar y visualizar el contenido de "caracteristicas_campeones.json"
with open("caracteristicas_campeones.json", "r") as file:
    champion_features = json.load(file)

# Cargar y visualizar el contenido de "arraysPartidasResultado.json"
with open("arraysPartidasResultado.json", "r") as file:
    match_results = json.load(file)

champion_dict = {champ['Key']: champ for champ in champion_features}
# Definir un vector de características neutrales
# Usaremos la media de cada atributo numérico para este vector
keys = ['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']
neutral_stats = {key: np.mean([champ[key] for champ in champion_features]) for key in keys}
neutral_stats['Name'] = 'Unknown'
neutral_stats['Key'] = -1  # Identificador especial para campeones desconocidos

# Función de transformación actualizada para manejar campeones no encontrados
def transform_match_updated(match, champion_dict, neutral_stats):
    result = match[0]
    # Utilizar estadísticas neutrales si el campeón no está en el diccionario
    blue_team = [champion_dict.get(champ_id, neutral_stats) for champ_id in match[1:6]]
    red_team = [champion_dict.get(champ_id, neutral_stats) for champ_id in match[6:11]]
    return result, blue_team, red_team

# Aplicar la transformación actualizada a la muestra para verificar
sample_transformed_updated = [transform_match_updated(match, champion_dict, neutral_stats) for match in match_results_sample]
sample_transformed_updated[0]  # Mostrar el resultado transformado de la primera partida


(1,
 [{'ID': 'Riven',
   'Key': 92,
   'Name': 'Riven',
   'Difficulty': 8,
   'HP': 630,
   'Armor': 33,
   'HPRegen': 8.5,
   'AttackDamage': 64,
   'AttackSpeed': 0.625},
  {'ID': 'Ivern',
   'Key': 427,
   'Name': 'Ivern',
   'Difficulty': 7,
   'HP': 655,
   'Armor': 27,
   'HPRegen': 7.0,
   'AttackDamage': 50,
   'AttackSpeed': 0.644},
  {'ID': 'Katarina',
   'Key': 55,
   'Name': 'Katarina',
   'Difficulty': 8,
   'HP': 672,
   'Armor': 28,
   'HPRegen': 7.5,
   'AttackDamage': 58,
   'AttackSpeed': 0.658},
  {'ID': 'Varus',
   'Key': 110,
   'Name': 'Varus',
   'Difficulty': 2,
   'HP': 600,
   'Armor': 27,
   'HPRegen': 3.5,
   'AttackDamage': 62,
   'AttackSpeed': 0.658},
  {'ID': 'Nautilus',
   'Key': 111,
   'Name': 'Nautilus',
   'Difficulty': 6,
   'HP': 646,
   'Armor': 39,
   'HPRegen': 8.5,
   'AttackDamage': 61,
   'AttackSpeed': 0.706}],
 [{'ID': 'Renekton',
   'Key': 58,
   'Name': 'Renekton',
   'Difficulty': 3,
   'HP': 660,
   'Armor': 35,
   'HPRegen': 8.0,
   

In [3]:

    all_matches_transformed = [transform_match_updated(match, champion_dict, neutral_stats) for match in match_results]
    
# Convertir las características de cada campeón en cada equipo en un solo vector para cada partida
    def flatten_match_data(match_data):
        result, blue_team, red_team = match_data
        # Aplanar las características de los campeones en un solo vector
        blue_features = [value for champ in blue_team for key, value in champ.items() if isinstance(value, (int, float))]
        red_features = [value for champ in red_team for key, value in champ.items() if isinstance(value, (int, float))]
        features = blue_features + red_features
        return result, features
    
    # Aplanar los datos para todos los partidos
    flattened_data = [flatten_match_data(match) for match in all_matches_transformed]
    
    # Extraer las características y los resultados
    X = np.array([features for _, features in flattened_data])
    y = np.array([result for result, _ in flattened_data])
    
    # Normalizar las características
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Muestra del resultado final procesado y normalizado
    X_scaled.shape, y.shape


((13783, 70), (13783,))

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Inicializar el clasificador KNN con un número arbitrario de vecinos, aquí elegimos k=5 para empezar
knn = KNeighborsClassifier(n_neighbors=5)

# Entrenar el modelo KNN
knn.fit(X_train, y_train)

# Predecir los resultados para el conjunto de prueba
y_pred = knn.predict(X_test)

# Calcular y mostrar métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["Perder", "Ganar"])

# Presentación de los resultados
print("Precisión del modelo:", accuracy)
print("Matriz de confusión:\n", conf_matrix)
print("Reporte de clasificación:\n", report)


Precisión del modelo: 0.6597751178817556
Matriz de confusión:
 [[ 750  494]
 [ 444 1069]]
Reporte de clasificación:
               precision    recall  f1-score   support

      Perder       0.63      0.60      0.62      1244
       Ganar       0.68      0.71      0.70      1513

    accuracy                           0.66      2757
   macro avg       0.66      0.65      0.66      2757
weighted avg       0.66      0.66      0.66      2757
